<a href="https://colab.research.google.com/github/Wejdan999/T55/blob/main/uesd_car.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Importing**


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split

In [ ]:
uc=pd.read_csv('/content/Used_Cars.csv')

### **Data View**

In [ ]:
uc.head()

In [ ]:
uc.sample(10)

In [ ]:
uc.info()

In [ ]:
uc.describe()

In [ ]:
uc.query('Seats==0')

### **Duplicate**


In [ ]:
uc.duplicated().sum()

In [ ]:
uc.drop_duplicates(inplace=True)

### ** Chicking Outlliers **

In [ ]:
sns.boxplot(uc[['Kilometers_Driven', 'Year', 'Price']]);

In [ ]:
Q1= uc['Kilometers_Driven'].quantile(0.25)
Q3=uc['Kilometers_Driven'].quantile(0.75)
IQR = Q3-Q1
lower_bound= Q1 - 1.5 *IQR
upper_bound= Q3 + 1.5 * IQR
uc = uc[(uc['Kilometers_Driven'] >= lower_bound ) & (uc['Kilometers_Driven']<= upper_bound)]


In [ ]:
sns.boxplot(uc[['Kilometers_Driven', 'Year', 'Price']]);

### **Correliation**

In [ ]:
uc['Engine']=uc['Engine'].str.replace('CC','')
print(uc['Engine'])

In [ ]:
uc['Power']=uc['Power'].str.replace('bhp','')
print(uc['Power'])

In [ ]:
uc['Mileage']=uc['Mileage'].str.replace('kmpl','')
print(uc['Mileage'])

In [ ]:
uc['Mileage']=uc['Mileage'].str.replace('km/kg','')
print(uc['Mileage'])

In [ ]:
uc['Power']=uc['Power'].str.replace('null','75')
print(uc['Power'])

In [ ]:
uc=uc.astype({'Engine':'float','Power':'float','Mileage':'float'})

In [ ]:
uc.info()

In [ ]:
uc.corr(numeric_only=True)

In [ ]:
sns.heatmap(uc.corr(numeric_only=True),annot=True)

There is a positive relationship between the engine , the power and the Price

In [ ]:
uc.head()

### **Dealing with nulls**

In [ ]:
uc.isnull().sum()

In [ ]:
uc= uc.drop('New_Price',axis=1)

In [ ]:
uc['Power'].fillna(uc['Power'].median, inplace=True)


In [ ]:
uc['Engine'].fillna(uc['Engine'].median, inplace=True)


In [ ]:
uc['Mileage'].fillna(uc['Mileage'].median, inplace=True)


In [ ]:
uc['Seats'].fillna(uc['Seats'].median, inplace=True)


In [ ]:
uc.isnull().sum()

### **X and Y **

In [ ]:
X = uc.drop(columns='Price')
y = uc[['Price']]

# **Data Splitting**

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
len(X_train)

In [ ]:
len(y_train)

In [ ]:
X_train.hist(figsize=(10,10),bins=50)

there is an outllier

In [ ]:
X_train.sort_values(by='Year', ascending=True)

In [ ]:
#X_train['Engine']=X_train['Engine'].str.replace('CC','')
#print(X_train['Engine'])

In [ ]:
#X_train['Power']=X_train['Power'].str.replace('bhp','')
#print(X_train['Power'])

In [ ]:
#X_train['Power'].mode()

In [ ]:
#X_train['Power']=X_train['Power'].str.replace('null','74')
#print(X_train['Power'])

In [ ]:
#X_train['Power'].isnull().sum()

In [ ]:
#X_train['Power'].fillna(uc['Power'].mode, inplace=True)

In [ ]:
#X_train['Power'].info()

In [ ]:
#X_train['Mileage']=X_train['Mileage'].str.replace('kmpl','')
#print(X_train['Mileage'])

In [ ]:
#X_train['Mileage']=X_train['Mileage'].str.replace('km/kg','')
#print(X_train['Mileage'])

In [ ]:
#X_train=X_train.astype({'Engine':'float','Power':'float','Mileage':'float'})

In [ ]:
X_train.corr(numeric_only=True)

In [ ]:
sns.heatmap(X_train.corr(numeric_only=True),annot=True)

In [ ]:
X_train.shape, y_train.shape

In [ ]:
X_train.head(10)

In [ ]:
X_train[['Fuel_Type']].value_counts()

In [ ]:
X_train.isna().sum()

In [ ]:
X_train.info()

In [ ]:
X_train[['Engine']].sample(10)

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.compose import ColumnTransformer

# Handle unknown categories by ignoring them
cat_pipe = make_pipeline(OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1))

num_pipe = make_pipeline(StandardScaler())

preproce_pipe = ColumnTransformer([("num", num_pipe, ['Engine', 'Power','Year']),
                                   ("cat", cat_pipe, ['Fuel_Type', 'Name', 'Transmission']),

                                  ])



In [ ]:
# Check for non-numeric values in the 'Engine' column
print(X_train['Engine'].apply(type).unique())

# Convert the 'Engine' column to numeric, handling errors
X_train['Engine'] = pd.to_numeric(X_train['Engine'], errors='coerce')

# Fill missing values in 'Engine' with the median
# You can also use other strategies like mean or a constant value.
X_train['Engine'] = X_train['Engine'].fillna(X_train['Engine'].median())

# Fit the model again
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(X_train[['Engine']], y_train)

In [ ]:
lr.score(X_train[['Engine']], y_train)

In [ ]:
lr.coef_

In [ ]:
lr.intercept_

In [ ]:
y_pred = lr.predict(X_train[['Engine']])
y_pred

In [ ]:
y_train

In [ ]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y_train, y_pred)